In [1]:
pip install kalshi-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
import kalshi_python
from pprint import pprint
import numpy as np
# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2 import ExchangeClient
import time
import json
import uuid
from kalshi_python.models import *
from datetime import datetime

In [2]:
# To start off, you need to have created an account at https://kalshi.com (Production) 
# or an account on the Demo https://demo.kalshi.co/

# prod_email = "" # change these to be your personal credentials
# prod_password = "" # (for extra security, we recommend using a config file)

demo_email = "Fill the email@gmail.com" # change these to be your personal credentials
demo_password = "Password" # (for extra security, we recommend using a config file)

# for prod
# prod_api_base = "https://trading-api.kalshi.com/trade-api/v2"

# for demo
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

## if wanting to test in prod:
# exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

## if wanting to test in demo
exchange_client = ExchangeClient(exchange_api_base = demo_api_base, email = demo_email, password = demo_password)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [3]:
config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://demo-api.kalshi.co/trade-api/v2'

# Create an API configuration passing your credentials.
# Use this if you want the kalshi_python sdk to manage the authentication for you.
kalshi_api = kalshi_python.ApiInstance(
    email='Sample Email',
    password='Sample Password',
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
print(exchangeStatus)

Exchange status response: 
{'exchange_active': True, 'trading_active': True}


In [10]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('predicted_temperature_max.csv')
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'], inplace=True)
# print(df)

df['Predicted_tempmax'] = np.round(df['Predicted_tempmax'],0)
df['Predicted_tempmax'] += 3
print(df)

# # Get the predicted temperature for Austin
# predicted_temp_for_austin = df.loc[df['Location'] == 'Austin', 'predicted_tempmax'].values[0]
# print(f"Predicted Temperature for Austin: {predicted_temp_for_austin}")


   Location  Predicted_tempmax
0    Austin               84.0
1   Chicago               44.0
2     Miami               86.0
3  New York               53.0


In [5]:
def determine_market_ticker_from_response(eventResponse, predicted_max_temp):
    for market in eventResponse.markets:
        subtitle = market.subtitle  # Adjusted based on your object structure
        if 'or below' in subtitle:
            max_temp = int(subtitle.split('°')[0])
            if predicted_max_temp <= max_temp:
                return market.ticker.split('-')[-1]
        elif 'or above' in subtitle:
            min_temp = int(subtitle.split('°')[0]) + 1
            if predicted_max_temp >= min_temp:
                return market.ticker.split('-')[-1]
        else:
            # Handling ranges correctly by cleaning the string of non-numeric before conversion
            temp_range = [int(temp.split('°')[0]) for temp in subtitle.split(' to ')]
            if temp_range[0] <= predicted_max_temp <= temp_range[1]:
                return market.ticker.split('-')[-1]
    return None


In [6]:
seriesTickers = {
    'New York': 'HIGHNY',
    'Chicago': 'HIGHCHI',
    'Austin': 'HIGHAUS',
    'Miami': 'HIGHMIA'
}
today = datetime.today()

# print(f"Processing cities: {df['Location'].tolist()}")

# for index, row in df.iterrows():
#     city = row['Location']
#     print(f"Processing {city}...")

# # Format the date as 'YYMMDD'
date_str = today.strftime('%y%b%d').upper()
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['Predicted_tempmax']
    seriesTicker = seriesTickers[city]
    

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)  # Replace with actual API call
    
    # Parse the event response to find matching market ticker
    # This code block assumes a function to parse and match the market ticker based on predicted temperature
    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    # Now you have the matching market ticker, you can retrieve the order book
    marketTicker = f'{eventTicker}-{matching_market_ticker}'
    orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)  # Replace with actual API call
    
    print(f'\nOrderbook for market: {marketTicker}')
    # Assuming pprint is imported or defined
    pprint(orderbookResponse)
# print(dir(orderbookResponse))



Orderbook for market: HIGHAUS-24APR03-T79
{'orderbook': {'no': [],
               'yes': [[23, 1000], [24, 1000], [25, 50], [50, 20], [99, 1]]}}

Orderbook for market: HIGHCHI-24APR03-B41.5
{'orderbook': {'no': [], 'yes': [[99, 1]]}}

Orderbook for market: HIGHMIA-24APR03-T84
{'orderbook': {'no': [], 'yes': [[25, 12], [99, 1]]}}

Orderbook for market: HIGHNY-24APR03-B50.5
{'orderbook': {'no': [], 'yes': [[97, 1000], [98, 1000], [99, 1]]}}


In [7]:
yes_offers = orderbookResponse.orderbook.yes
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']


In [8]:
orderbook_dict = orderbookResponse.to_dict()
yes_offers = orderbook_dict['orderbook']['yes']
for index, row in df.iterrows():
    city = row['Location']
    predicted_max_temp = row['Predicted_tempmax']
    seriesTicker = seriesTickers[city]

    eventTicker = f'{seriesTicker}-{date_str}'
    eventResponse = kalshi_api.get_event(eventTicker)

    matching_market_ticker = determine_market_ticker_from_response(eventResponse, predicted_max_temp)
    
    if matching_market_ticker:
        marketTicker = f'{eventTicker}-{matching_market_ticker}'
        orderbookResponse = kalshi_api.get_market_orderbook(marketTicker)
        
        # Convert the orderbookResponse to a dictionary to access 'yes' and 'no' offers
        orderbook_dict = orderbookResponse.to_dict()
        yes_offers = orderbook_dict['orderbook']['yes'] if 'yes' in orderbook_dict['orderbook'] else []
        
        if yes_offers:
            best_yes_offer = min([offer[0] for offer in yes_offers])
            
            if exchangeStatus.trading_active:
                orderUuid = str(uuid.uuid4())
                orderResponse = kalshi_api.create_order(CreateOrderRequest(       
                    ticker=marketTicker,
                    action='buy',
                    type='limit',
                    yes_price=best_yes_offer - 1,  # Adjust based on your strategy
                    count=1000,  # And your desired amount
                    client_order_id=orderUuid,
                    side='yes',
                ))
                print(f'\nOrder submitted for {city}: ')
                pprint(orderResponse)
        else:
            print(f'\nNo "yes" offers available for {city}, no orders will be placed.')
    else:
        print(f'No matching market ticker found for {city} based on the predicted temperature.')
 


Order submitted for Austin: 
{'order': {'action': 'buy',
           'client_order_id': 'ad2cc17d-d850-4b43-9afd-2d49319a5128',
           'created_time': '2024-04-04T00:09:28.027695Z',
           'expiration_time': None,
           'no_price': 78,
           'order_id': '54b56f18-6708-45f7-a55a-429accb379c5',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHAUS-24APR03-T79',
           'type': 'limit',
           'user_id': 'dbad063d-b115-48bd-a024-079c02ba5ad4',
           'yes_price': 22}}

Order submitted for Chicago: 
{'order': {'action': 'buy',
           'client_order_id': '15d074d4-e934-4f87-a54f-d1c67ab69447',
           'created_time': '2024-04-04T00:09:28.376481Z',
           'expiration_time': None,
           'no_price': 2,
           'order_id': '0558695d-f6b7-4e28-80f4-91b0d38047b8',
           'side': 'yes',
           'status': 'resting',
           'ticker': 'HIGHCHI-24APR03-B41.5',
           'type': 'limit',
           'user_id': '